# <center>CS559 - Lecture 3: Unsupervised Learning </center>
<center><b>Spring 2022</b></center>

<p><a name="Outline"></a></p>

# Outline
1. <a href="#RecapFromLastLecture">Recap From Last Lecture </a><br>
2. <a href="#Introduction to Unsupervised Learning">Introduction to Unsupervised Learning</a><br>
3. <a href="#Cluster Analysis">Cluster Analysis</a><br>
    1. <a href="#Kmeans">K-means</a><br>
    2. <a href="#Hierarchical">Hierarchical</a><br>
4. <a href="#PCA">Dimentional Reduction: Principal Component Analysis (PCA)</a><br>

<p><a name="RecapFromLastLecture"></a></p>

# 1. Recap From Last Lecture
## Machine Learning Overview
- Machine Learning
- Learning - Supervised Learning, Unsupervised Learning, Reinforcement Learning
- Preprocessing - EDA, Scaling, Missing value imputation, Vectorizing categorical data, and Feature Engineering
- Modeling - Data Split
    - Training and Test sets
    - Traning, Validation, and Test sets
    - Cross Validation
        - Heldout, K-Fold, etc... 

<p><a name="Introduction to Unsupervised Learning"></a></p>

# 2. Introduction to Unsupervised Learning
- Only a set of N observations with p features and no reponse (target) variables.
    - Unlabeled data which is much easier to obtain... 
- Goal: to infer the properties directly without knowing the "correct answers or the error for each observation. 
    - "<b>Learning Without Teacher</b>" - No direct measure of success.
    - Therefore, more subjective than supervised learning.
- Two methods:
    - **Clustering**: a broad class of methods for grouping or segmenting a collection of objects into distinct subjects known as **clusters**.
        - example: groups of online shoppers characterized by their browsing and purchase histories.
    - **Dimension Reduction**: a method to reduce the dimension of data while keeping the most of information. 
        - often used for data visualization or data preprocessing for supervised learning.

# 3. Cluster Analysis
- Task: aim to uncover <b>underlying structure</b> of the data and see what pattern exists in the data. 
    - We aim to group together observations that are similar while separating observations that are dissimilar. 
- Cluster analysis attempts to explore possible subpopulations that exist within your data.
- Typical questions that cluster analysis attempts to answer are: 
    - Approximately how many subgroups exist in the data? 
    - Approximately what are the sizes of the subgroups in the data? 
    - What commonalities exist among members in similar subgroups? 
    - Are there smaller subgroups that can further segment current subgroups? 
    - Are there any outlying observations? 
        - Notice that these questions are largely exploratory in nature. 
        
## 3.A K-Means Clustering
- With the K-means clustering algorithm, we aim to split up our observations into a predetermined number of clusters.
    - The number of clusters K must be specified in advance.
    - These cluster memberships are distinct and non-overlapping.
- The data points in each of the clusters are determined to be mostly similar to a specific centroid value:
    - The centroid of a cluster represents the average of the observations within a given cluster; it is a single theoretical center that represents the prototypical member that exists within the given cluster. 
    - Each observation will be assigned to exactly one of the K clusters depending on where the observation falls in the feature space relative to the cluster centroid locations. 

In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8, 6
np.random.seed(42)
x1 = np.random.randn(100, 2) * 7 + 10 
x2 = np.random.randn(100, 2) * 7 - 10 
x = np.row_stack([x1, x2])
plt.scatter(x[:, 0], x[:, 1])
plt.show()

- A simulated data set with 150 observation in 2-dimensional space.

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
for i,ax in zip(range(2,5),[ax1, ax2, ax3]):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(x)
    ax.scatter(x[:, 0], x[:, 1],c=kmeans.labels_, alpha=0.8)
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('$k=$'+str(i))
plt.subplots_adjust(left=0.1,bottom=0.1,right=1,top=0.9,wspace=0.4,hspace=0.4)
plt.show()

- The color labels the cluster to which it has been assigned. Note that the cluster coloring is arbitrary since there is no absolute ordering of the clusters. 
- Now the main question: what technique does k-means algorithm use to create these clusters? 
    - Suppose we use the **Euclidean** distance 
    $$D(\vec{q},\vec{p})=\sqrt{\sum_{i=1}^N(q_i-p_i)^2}.$$ 
    - Then the **within-cluster variation** $W(C_k)$ is defined as: 
$$W(C_k)=\frac{1}{|C_k|}\sum_{i,i'\in C_k}\sum_{j=1}^p (x_{ij}-x_{i'j})^2$$
    - where 
        - $C_k$ is the total number of observations in cluster $k$
        - $i,i'\in C_k$ are indices of observations in cluster $C_k$
        - $p$ is the number of variables/features in dataset.
- Since the within-cluster variation is a quantitative gauge of the amount by which the observations in a specific cluster differ from one another, we want to **minimize** the sum of this quantity $W(C_k)$ over all clusters: 
$$\min_{C_1,\dots,C_k}\Big\{\sum_{i=1}^k W(C_i)\Big\}$$
- In other words, we would like to partition the observations into K clusters such that the total within-cluster variation aggregated across all K clusters is as small as possible; the optimization problem for K-means is as follows:
$$\min_{C_1,\dots,C_k}\Big\{\sum_{i=1}^k \frac{1}{|C_k|}\sum_{i,i'\in C_k}\sum_{j=1}^p (x_{ij}-x_{i'j})^2\Big\}$$
- To find the global minimum of the above function is very difficult.
- In practice, most K-means packages perform the following greedy algorithm, also known as Lloyd algorithm in the computer science circle:
    1. Randomly assign  an integer label, from 1 to K (where K is the number of clusters), to each of the observations. These serve as initial cluster assignments for the observations. 
    2. Iterate until the cluster assignments stop changing:
        1. For each of the K clusters, compute the cluster’s new centroid. 
        2. Assign each observation to the cluster whose centroid is closest (closest is measured using Euclidean distance). 

In [ ]:
np.random.seed(123)
x3 = np.random.randint(2, size=len(x1))
x=pd.DataFrame(np.column_stack([x1,x2,x3]))
x

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))

x=np.row_stack([x1,x2])
ax1.scatter(x[:,0], x[:,1])
ax1.set_title('1. Data')

np.random.seed(123)
L=4
x3 = np.random.randint(L, size=len(x))
x=pd.DataFrame(np.column_stack([x,x3]))
x.columns = ['x','y','label']



x_mean=x.groupby('label').mean()
x_mean.reset_index(inplace=True)

for l,color in zip(range(0,L),['blue','red','green','yellow']):
    ax2.scatter(x['x'][x['label']==l], x['y'][x['label']==l],alpha=0.5,color=color,label='label'+str(l))
    ax2.legend()
    ax2.set_title('2. Random Label Initialization')
    ax3.scatter(x['x'][x['label']==l], x['y'][x['label']==l],alpha=0.5,color=color,label='label'+str(l))
    ax3.scatter(x['x'][x['label']==l].mean(),x['y'][x['label']==l].mean(), marker="o", 
                s=300,alpha=1,color=color,edgecolor='black')
    ax3.legend()
ax3.set_title('3. Centroids with the initial labels')
plt.subplots_adjust(left=0.1,bottom=0.1,right=1,top=0.9,wspace=0.4,hspace=0.4)
plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,5))
kmeans = KMeans(n_clusters=L,random_state=0,n_init=1,max_iter=1)
kmeans.fit(x[['x','y']])
x['newlabel']=kmeans.labels_
x['newlabel'].replace({3:1,0:2,2:0,1:3},inplace=True)
for l,color in zip(range(0,L),['blue','red','green','yellow']):
    ax1.scatter(x['x'][x['newlabel']==l], x['y'][x['newlabel']==l],alpha=0.5,color=color,label='new label'+str(l))
    ax1.legend()
    ax1.set_title('4. Cluster Respects to the Current Centroids')

    ax2.scatter(x['x'][x['newlabel']==l], x['y'][x['newlabel']==l],alpha=0.5,color=color,label='new label'+str(l))
    ax2.scatter(x['x'][x['newlabel']==l].mean(),x['y'][x['newlabel']==l].mean(), marker="o", 
            s=300, color=color,alpha=1,edgecolors='black')
    ax2.set_title('5. Find New Centroids')
    ax2.legend()

kmeans = KMeans(n_clusters=L,random_state=0,n_init=1,max_iter=10)
kmeans.fit(x[['x','y']])
x['newlabel']=kmeans.labels_
x['newlabel'].replace({3:1,0:2,2:0,1:3},inplace=True)

for l,color in zip(range(0,L),['blue','red','green','yellow']):
    ax3.scatter(x['x'][x['newlabel']==l], x['y'][x['newlabel']==l],alpha=0.5,color=color,label='new label'+str(l))
    ax3.scatter(x['x'][x['newlabel']==l].mean(),x['y'][x['newlabel']==l].mean(), marker="o", 
            s=300, color=color,alpha=1,edgecolors='black')
    ax3.set_title('6. Final Results')
    ax3.legend()

plt.show()

- The K-means procedure always converges:
    1. If you run the algorithm from a fixed initial assignment, it will reach a stable endpoint where the clustering solution will no longer change through the iterations. 
    2. Unfortunately, the guaranteed convergence is to a local minimum. 
        - Thus, if we begin the K-means algorithm with a different initial configuration, it is possible that convergence will find different centroids and therefore ultimately assigning different cluster memberships. 
- What can we do to get around this?
    - Run the K-means procedure several times and pick the clustering solution that yields the smallest aggregate within-cluster variance. 
- The Kmeans++ (2007) improves the random seeding of the original KMeans.
    1. The initialization step runs inductively.
    2. Firstly, pick a data point randomly as the first centroid.
    3. Suppose that k of the seed centroid have been chosen, compute for each data point x the distance $D(x)$ to the closest centroid among these k seed centroids. Select the $(k+1)^{th}$ centroid randomly, according to a probability distribution with probability proportional to $D(x)^2$.
    4. In each inductive step, the newly found seed centroid trends to keep a far distance from the existing ones.
    5. The algorithm will stop when it has found the least/best (local optimum) value. 
- The default initialization scheme of Scikit-Learn’s KMeans uses KMeans++. 

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(15,10))
max_iter = 10
n_init=1
for i in range(0,2):
    for j in range(0,3):
        n_init=np.random.randint(n_init,5)
        max_iter = np.random.randint(max_iter,100)
        kmeans = KMeans(n_clusters=4,n_init=n_init,max_iter=max_iter)
        kmeans.fit(x[['x','y']])
        axs[i,j].scatter(x['x'],x['y'],c=kmeans.labels_,alpha=0.8)
        axs[i,j].set_title('inertia='+str(round(kmeans.inertia_,2))+', n_iter='+str(kmeans.n_iter_))
plt.show()

In [ ]:
def plot_inertia(km, X, n_cluster_range):
    inertias = []
    for i in n_cluster_range:
        km.set_params(n_clusters=i)
        km.fit(X)
        inertias.append(km.inertia_)
    plt.plot(n_cluster_range, inertias, marker='o')
    plt.title('Elbow method')
    plt.xlabel('Number of clusters')
    plt.ylabel('Inertia')
    plt.show()

In [ ]:
plot_inertia(KMeans(), x, range(1,10))

In [ ]:
from __future__ import print_function
from sklearn.cluster import KMeans
kmeans = KMeans()

**Arguments**:
- n_clusters: The number of clusters to divide, default is 8.
- max_iter: The maximal number of iterations, default is 300.
- n_init: Number of time the k-means algorithm will run with different centroid seeds. The final results will be the best output of n_init consecutive runs in terms of inertia. default is 10.
- init: Method for initialization, defaults to 'k-means++'. Other options are 'random' or an ndarray of shape (n_clusters, n_features) and gives the initial centers.
- random_state: Optional. The generator used to initialize the centers. If an integer is given, it fixes the seed. Defaults to the global numpy random number generator.
Usually, we just need to set the argument n_clusters to determine how many groups are we going to split.

**Attributes**:
- cluster_centers_: The coordinates of cluster centers.
- labels_: Labels of each observation, which indicate the group number of each observation.
- inertia_: Sum of distances of samples to their closest cluster center.

The most import attribute here is the labels_ .

**Methods**:
- fit: Fit k-means clustering on a given data set.
- fit_predict: Compute cluster centers and predict cluster index for each sample.
- get_params: Get parameters for this estimator.
- set_params: Set the parameters of this estimator.
- predict: Given a set of data, predict the closest cluster each sample belongs to.

In this case, we try to split the iris data into multiple groups by using the features sepal length, sepal width, petal length, petal width.
- Set the argument n_clusters to 3.
- Fit the iris data.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
kmeans = KMeans()

In [ ]:
plot_inertia(kmeans, iris.data, range(1, 10))

In [ ]:
kmeans.set_params(n_clusters=3)
kmeans.fit(iris.data)

- The label of each observation.

In [ ]:
kmeans.labels_

- The centroid of each cluster:

In [ ]:
kmeans.cluster_centers_

In [ ]:
import  matplotlib.pyplot as plt
plt.scatter(iris.data[:, 2], iris.data[:, 3], c=kmeans.labels_, alpha=0.8)
plt.scatter(kmeans.cluster_centers_[:, 2], kmeans.cluster_centers_[:, 3], marker="+", s=1000, c=[0, 1, 2])
plt.xlabel('Petal Length')
plt.ylabel('Petal Width')
plt.show()

In [ ]:
plt.scatter(iris.data[:, 0], iris.data[:, 1], c=kmeans.labels_, alpha=0.8)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker="+", s=1000, c=[0, 1, 2])
plt.xlabel('Sepal Length')
plt.ylabel('Sepal Width')
plt.show()

- The big markers "+" refers to the centroid of each cluster.
- We can also fit the principal components to the KMeans algorithm. Perform K means on the new dataset below, and then find the centers and the labels.

In [ ]:
from matplotlib import cm
from sklearn.metrics import silhouette_samples
import numpy as np

def plot_silhouette(km, x):
    y_km = kmeans.fit_predict(x)
    cluster_labels = np.unique(y_km)
    n_clusters = cluster_labels.shape[0]
    silhouette_vals = silhouette_samples(x, y_km, metric='euclidean')
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    for i, c in enumerate(cluster_labels):
        # Aggregate the silhouette scores for samples belonging to
        # cluster c, and sort them
        c_silhouette_vals = silhouette_vals[y_km == c]
        c_silhouette_vals.sort()

        size_cluster_c = len(c_silhouette_vals)
        y_ax_upper += size_cluster_c
        color = cm.jet(i*1.0/n_clusters)
        plt.barh(range(y_ax_lower, y_ax_upper), c_silhouette_vals, height=1.0, color=color)

        # Compute the new y_ax_lower for next plot
        yticks.append((y_ax_lower + y_ax_upper) / 2)
        y_ax_lower += size_cluster_c

    # The vertical line for average silhouette score of all the values
    silhouette_avg = np.mean(silhouette_vals)
    plt.axvline(silhouette_avg, color='red', linestyle='--')

    plt.yticks(yticks, cluster_labels + 1)
    plt.title('Silhouette Analysis')
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette coefficient')
    plt.show()

In [ ]:
kmeans.set_params(n_clusters=3)
plot_silhouette(kmeans, iris.data)

- The silhouette value is a measure of how similar an object is to its own cluster (cohesion) compared to other clusters (separation). The silhouette ranges from −1 to +1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.
- If most objects have a high value, then the clustering configuration is appropriate. If many points have a low or negative value, then the clustering configuration may have too many or too few clusters.

### Summary of K-means
K-means clustering algorithms require:
1. the choice of the number of classes to be clustered, 
2. a starting cluster configuration assignment. 
3. In most cases, this is hard to determine. 

#### Strengths
- Use simple principles for identifying clusters which can be explained in non-statistical terms.
- It is fairly efficient and performs well at dividing the data into useful clusters.

#### Weaknesses
- It is less sophisticated than more recent clustering algorithms.
- Because it uses initial random choice, it is not guaranteed to find the optimal set of clusters.
- Require a reasonable guess for how many clusters naturally exist in the data.


## 3.b Hierarchical Clustering (Agglomerative Clustering)

**Hierarchical clustering** method is another popular clustering method which seeks to build a hierarchy of clusters. 
- It does not require the user to specify the number of clusters. Instead, it requires to measure the dissimilarity between the pairs of clusters. 
- The clusters at a higher level are created by merging clusters at the lower level:
    - At the lowest level, each cluster contains a single observations.
    - At the highest level, all of the data points from a single cluster. 

- Two strategies for hierarchical clustering: bottom-up and top-down. 
- The approach can be summarized as:
    1. Start with each point in its own cluster.
    2. Identify the two clusters which are most similar and merge them.
    3. Repeat step 2.
    4. Ends when all data points are in a single cluster. 

In the lecture we show how to build a hierarchy in a bottom-up fashion.
<img src="img/hierarchical_01.png" width=600 length=700 />
- There are some interpretative advantages to visualizing the dendrogram created from hierarchical clustering:
    - The lower down in the dendrogram a cluster fusion occurs, the more similar the fused clusters are to each other.
    - The higher up in the dendrogram a fusion occurs, the more dissimilar the fused groups are to each other.
- In general, for any two observations we can inspect the dendrogram and find the location at which the groups that contain those two observations are fused together to get an idea of their dissimilarity.
    - Be careful to consider the groups of points in the fusions within the dendrograms, not just individual points.
<img src="img/hierarchical_02.png" width=600 length=800 />
- Begin with n observations and a distance measure of all pairwise dissimilarities. At this step, treat each of the n observations as their own clusters.
    - For $i=n,n-1,\dots,2$:
        1. Evaluate all pairwise inter-cluster dissimilarities among the i clusters and fuse together the pair of clusters that are the least dissimilar.
        2. Note the dissimilarity between the recently fused cluster pair and mark that as the associated height in the dendrogram.
        3. Repeat the process in step 1, calculating the new pairwise inter-cluster dissimilarities among the remaining $(i - 1)$ clusters.
- While we do not need to specify K a priori, in order to perform hierarchical clustering there are a few choices we need to make. Particularly:
    - A dissimilarity measure.
    - A linkage method.
- We are already familiar with the idea of choosing a dissimilarity measure with the choice of distance metric. In many cases, it is sufficient to use the Euclidean distance.
- A **linkage** is a measure of the dissimilarity between two group of points. So far we only define the distance between two points, but what do we do when we want to assess the similarity among two groups of points?
- The most common types of linkage are described below.
     - First, compute all pairwise dissimilarities between the observations in cluster A and the observations in cluster B. Then:
      1. Complete Linkage: Maximal inter-cluster dissimilarity.
            - Record the largest of the dissimilarities listed between members of A and of B as the overall inter-cluster dissimilarity.
            - sensitive to outliers, yet it tends to identify clusters that are compact, somewhat spherical objects with relatively similar diameters.
      2. Single Linkage: Minimal inter-cluster dissimilarity.
             - Record the smallest of the dissimilarities listed between members of A and of B as the overall inter-cluster dissimilarity.
             - not as sensitive to outliers, yet tends to identify clusters that have a chaining effect; these clusters often fail to represent intuitive groupings among our data, and the observations in the same cluster might be quite distant from one another.
      3. Average Linkage: Mean inter-cluster dissimilarity.
            - Record the average of the dissimilarities listed between the members of A and of B as the overall inter-cluster dissimilarity.
            - tends to strike a balance between the pros and cons of complete linkage and single linkage.

       4. Ward’s Linkage: Minimum variance method (for Euclidean distance).
            - Minimize the variance of the clusters being merged.
<img src="img/linkage.png" width=800 length=800 />



In [ ]:
from sklearn.cluster import AgglomerativeClustering
hier = AgglomerativeClustering()

**Arguments**:
- n_clusters: The number of clusters to find. default=2
- affinity: Metric used to compute the linkage. Can be “euclidean”, “l1”, “l2”, “manhattan”, “cosine”. If linkage is “ward”, only “euclidean” is accepted. default: “euclidean”
- linkage: Which linkage criterion to use. The linkage criterion determines which distance to use between sets of observation. The algorithm will merge the pairs of cluster that minimize this criterion.
    - ward minimizes the variance of the clusters being merged.
    - average uses the average of the distances of each observation of the two sets.
    - complete or maximum linkage uses the maximum distances between all observations of the two sets.

**Arguments**:
- The possible values of the affinity are “euclidean”, “l1”, “l2”, “manhattan”, “cosine”.
- "l1" is the same as "manhattan", while "l2" is the same as "euclidean".
- "cosine" here in python is not the same as we told previously.
The smaller the euclidean/manhattan distance is, the closer the two observations are. The smaller the cosine is, the more far the observations are.
So in Python, the cosine distance is redefined as :
$$1-\frac{\sum_{i=1}^n x_{1i}\times x_{2i}}{\sqrt{||x_1||^2}\times\sqrt{||x_2||^2}}$$
Now the cosine distance ranges from 0 to 2, and the smaller it is, the closer the observations are.
- 0: two vectors point to the same direction
- 1: perpendicular
- 2: opposite direction

#### Distance in Scikit Learn
We can compute the different distances by using the function pairwise_distances.

In [ ]:
import numpy as np
a = np.array([[1, 2], [2, 1]])

##### Euclidean Distance

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
pairwise_distances(a, metric='l2') 

In [ ]:
((1-2)**2 + (2-1) ** 2) ** 0.5

##### Cosine Distance

In [ ]:
pairwise_distances(a, metric='cosine') 

In [ ]:
1 - (1*2 + 2*1)/(5**0.5 * 5**0.5)

**Attributes**:
- labels_: Cluster labels for each observation.
- n_leaves_: Number of leaves in the hierarchical tree, which is also the number of observations.
**Methods**:
- fit: Fit the hierarchical clustering on the data.
- get_params: Get parameters for this estimator.
- set_params: Set the parameters of this estimator.

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
hier.set_params(n_clusters=3)
hier.fit(iris.data)

In [ ]:
hier.labels_

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8, 6
plt.scatter(iris.data[:, 2], iris.data[:, 3], c=hier.labels_, alpha=0.8)
plt.show()

In [ ]:
hier.set_params(n_clusters=3, affinity='cosine', linkage='complete')
hier.fit(iris.data)
label = hier.labels_
plt.scatter(iris.data[:, 2], iris.data[:, 3], c=label, alpha=0.8)
plt.xlabel('Expensure')
plt.ylabel('Transfer')
plt.show()

In [ ]:
hier.set_params(n_clusters=3, affinity='euclidean', linkage='average')
hier.fit(iris.data)
label = hier.labels_
plt.scatter(iris.data[:, 2], iris.data[:, 3], c=label, alpha=0.8)
plt.xlabel('Expensure')
plt.ylabel('Transfer')
plt.show()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
import pandas as pd
def linkage_frame(data):
    row_clusters = linkage(data, method='complete', metric='euclidean')
    columns = ['row label 1', 'row label 2', 'distance', 'no. items in clust.']
    index = ['cluster %d' % (i + 1) for i in range(row_clusters.shape[0])]
    linkage_df = pd.DataFrame(row_clusters, columns=columns, index=index)
    return linkage_df

In [ ]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df.head()

In [ ]:
linkage_df = linkage_frame(iris_df.values)
linkage_df.head()

In [ ]:
dendrogram?

In [ ]:
row_dendr = dendrogram(linkage_df, leaf_rotation=90, leaf_font_size=8)
plt.tight_layout()
plt.ylabel('Euclidean distance')
plt.show()

In [ ]:
row_dendr = dendrogram(linkage_df, leaf_rotation=90, truncate_mode='level', p = 3, leaf_font_size=8)
plt.tight_layout()
plt.ylabel('Euclidean distance')
plt.show()

<p><a name="PCA"></a></p>

# 4. Dimentional Reduction: Principal Component Analysis (PCA)

**Multicollinearity** is a phenomenon in which two or more predictor variables in a multiple regression model are highly correlated, meaning that one can be predicted from the others through linear formulae with a substantial degree of accuracy.

Issues:
- The regression coefficients of highly correlated variables might be inaccurate (high model variance).
- The estimate of one variable's impact on the dependent variable Y while controlling for the others tends to be less precise.
- The nearly collinear variables contain similar information about the dependent variable, which may lead to overfitting.
- The standard errors of the affected coefficients tend to be large.
- Given a number of observations, additional dimensions spread the points out further and further from one another.
- Sparsity becomes exponentially worse as the dimensionality of the data increases.
- The model Supportive Vector Machine (lecture 6) takes advantage of the curse of dimensionality.
<img src="img/pca_01.png" width=500 length=700 />

**Principal component analysis (PCA)** is a tool that finds a sequence of linear combinations of the variables to convert a set of observations of possibly correlated variables into a set of values of linearly uncorrelated variables called principal components.
- Ideal input variables: 
    - Linearly uncorrelated
    - Low-dimensional in the feature space
## Motivation

In [ ]:
### Loading Packages
import numpy as np
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
np.random.seed(3)
x = np.random.rand(5)
y1  = np.array([0.2]*5)
y2 = np.random.rand(5) * 0.1 + 0.2
for ax,y in zip([ax1,ax2],[y1,y2]):
    ax.scatter(x,y)
    ax.hlines(y=0.2,xmin=0.2,xmax=1, linestyle='--',alpha=0.3)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_ylim(-0, 1)
    ax.set_xlim(0.2, 1)

plt.subplots_adjust(left=0.1,bottom=0.1,right=1,top=0.9,wspace=0.4,hspace=0.4)
plt.show()

- Left: We always do not need all the features. The y component of all the points are the same, it provides NO additional information.
- Right: y values are restricted in a much smaller region than x values. This suggests that x component might provide more information. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))

for i in range(len(x)):
    ax1.plot([x[i], x[i]], [y[i], 0], color='red', alpha=0.3)
ax1.scatter(x, y)
ax1.scatter(x, np.zeros(len(x)), color='red')
ax1.set_ylim(-0, 1)
ax1.set_xlim(0.2, 1)
ax1.set_xlabel('x')
ax1.set_ylabel('y')


for i in range(len(x)):
    ax2.plot([x[i], 0], [y[i], y[i]], color='red', alpha=0.3)
ax2.scatter(x, y)
ax2.scatter(np.ones(len(x)) * 0.2, y, color='red')
ax2.set_ylim(-0, 1)
ax2.set_xlim(0.2, 1)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
plt.subplots_adjust(left=0.1,bottom=0.1,right=1,top=0.9,wspace=0.4,hspace=0.4)
plt.show()

- One way to compare is to project the observations to the two axes.
    - Left: For `x` axis, the projection spreads in a range around 0.3 to 0.9.
    - Right: In contrast, `y` is restricted in a much smaller region. This suggests that `x` component might provide more information, because all the `y` components are "about the same".

- Consider a set of 30 points in a three dimensional space. In such a scenario, we have: 20 observations and 3 features. 

In [ ]:
def rotate(array):
    data = [
        [1, 0, 0],
        [0, np.sqrt(3) / 2, -np.sqrt(1) / 2],
        [0, np.sqrt(1) / 2, np.sqrt(3) / 2]]
    rot = np.matrix(data=data).T
    return np.array(np.matrix(array) * rot)

n = 30
np.random.seed(108)
z = 10. * np.random.rand(n) - 5
theta = 2 * np.pi * np.random.rand(n)
a = 5 - np.abs(z)
x = a / 2.5 * np.cos(theta)
y = a / 5. * np.sin(theta)
data = np.zeros((n, 3))
data[:, 0] = x
data[:, 1] = y
data[:, 2] = z
data = rotate(data)

In [ ]:
def plot_origin():
    ax.scatter(0, 0, 0, marker='o', s=26, color='black', alpha=1)
    
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.text(0+0.1,0+0.1,0+0.1,'origin')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

- We compare the importance of each direction. Note that the chosen directions in the example are not parallel to any coordinate axis. 

In [ ]:
pca = PCA()
pca.fit(data)
first = pca.components_[[0]]
second= pca.components_[[1]]
third = pca.components_[[2]]
raw_data = data
data = data - pca.mean_

In [ ]:
def plot_vec(array, length, color='blue', alpha=1):
    kwargs = dict(
        color=color,  # color of the curve
        linewidth=1.4,  # thickness of the line
        # linestyle='--',  # available styles - -- -. :
        alpha=alpha,
    )
    ax.plot(*zip(-array[0] * length, array[0] * length), **kwargs)

fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()
plot_vec(first, 6)

arbi = np.array([[ 0.99249426,  0.05066054,  0.1113043 ]])
plot_vec(arbi, 2, color='orange')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show() 

- We project each observation orthogonally to the “orange” direction. 

In [ ]:
def project2vec(data, vec, id_=0, color='green', along=False):
    pp = data[[id_]]
    proj = (np.sum(vec*pp)*vec)
    ax.scatter( *( proj.ravel() ), color=color, s=16)
    kwargs = dict(
        color=color,  # colour of the curve
        linewidth=1.4,  # thickness of the line
        # linestyle='--',  # available styles - -- -. :
        alpha=0.5,
    )
    ax.plot(*(zip(pp[0], proj[0])), **kwargs)
    if along:
        along_kwargs = dict(
            color='Dark' + color,  # colour of the curve
            linewidth=1.4,  # thickness of the line
            # linestyle='--',  # available styles - -- -. :
            alpha=1,
        )
        ax.plot(*(zip(np.array([0,0,0]), proj[0])), **along_kwargs)
    return np.sum(vec*pp)

fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()
plot_vec(first, 6)


plot_vec(arbi, 2, color='orange')


len_proj = project2vec(data, arbi, id_=13, color='orange', along=False)


ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_vec(first, 6)


plot_vec(arbi, 2, color='orange')

proj_arbi = []
for i in range(n):
    len_proj = project2vec(data, arbi, id_=i, color='orange', along=False)
    proj_arbi.append(len_proj)


ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()


- We project each observation orthogonally to the “blue” direction. 

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_vec(first, 6)


plot_vec(arbi, 2, color='orange')

proj_first = []
for i in range(n):
    len_first = project2vec(data, first, id_=i, color='blue', along=False)
    proj_first.append(len_first)


ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

The "blue" direction above is actually the **first principal component**, which means:

- it is the direction on which the projection of the observations is more widely spread than the projetion on any other direction.

- being a direction (vector), it has as many components as the number of the features usually.

Note that we characterized the first principal component direction, but we didn't show how it's found. We will discuss that but if you don't care about math, Python will find it for you.

In [ ]:
fig = plt.figure(figsize=(15, 5))
plt.scatter(proj_first, np.ones(n) * 2, color='blue')
plt.axhline(y=2, color='blue')
plt.scatter(proj_arbi, np.ones(n), color='orange')
plt.axhline(y=1, color='orange')
plt.ylim(0, 3)
plt.yticks([])
plt.show()

- The statements above charaterizes the proncipal component direction. To find the principal component direction, we need to apply **linear algebra** which we will discuss later. However, if you don't care about math, Python will find it for you.

- With the first loading vector (heuristically the most important one), we want to keep, for all the observations, only the information recorded in this direction.
    - This is done by orthogonal linear projection.
    - There are in general N (the number of samples) components for principal component.
    - There are in general p (the number of features) components for a principal direction (the loading vector). 
    - The principal components live in the space of samples, while the principal directions live in the space of features. 


### First Principal Component

- The first loading vector gives us a vector of length 30. This vector is the first principal component. 
- We need 30 records for all the observations.
- We do not use the xyz-coordinates – we use one coordinate, the first principal component. 
- The red projection can describe a certain length away from the origin along the principal direction and is a vector in the original xyz-coordinates. 

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()
plot_vec(first, 6, alpha=0.4)

proj_len = project2vec(data, first, id_=3, color='red', along=True)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=0.2)
plot_origin()
plot_vec(first, 6, alpha=0.2)
ax.scatter(*(proj_len*first[0]), marker='o', s=16, color='red', alpha=1)
ax.plot(*(zip(np.array([0,0,0]), proj_len*first[0])), 
        color='red', linewidth=1.4, alpha=1)

pca_fmt = '%.2f units away from the origin\n along the 1st pca direction'
pca_coord_txt = pca_fmt % proj_len 
ax.text(-0.01044 + 0.2, 1.164, -1.585, pca_coord_txt, color='green', size=12)

origin_fmt = '(%.2f, %.2f, %.2f)\nin the x-y-z coordinate)'
origin_coord_txt = origin_fmt %tuple((proj_len*first[0]))
ax.text(-0.01044 - 1.8, 0.964 - 1, -1.585 - 1, origin_coord_txt, color='blue', size=12)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

### Second Principal Component
- The information stored is about the variation of the points across the whole sample set.
- Not all directions are born equal. 
- The first principal component provides the most information but most likely not all. 
- We remove the data information stored in the first principal component.
- Then find the new direction (orthogonal to the original principal direction) on which the projection of the observations is most widely spread. 
- We consider the 2-D plane that is perpendicular to the first loading vector. 


In [ ]:
def plot_plane(normal, color='blue', alpha=0.2, x_min=-1.5, x_max=2.5, y_min=-2.5, y_max=1.5):
    x_min_rng = list(range(int(np.floor(x_min) + 1), 0))
    x_max_rng = list(range(int(np.floor(x_max))))
    y_min_rng = list(range(int(np.floor(y_min) + 1), 0))
    y_max_rng = list(range(int(np.floor(y_max))))
    surf_x, surf_y = np.meshgrid(
        [x_min] + x_min_rng + x_max_rng + [x_max],
        [y_min]+ y_min_rng + y_max_rng + [y_max])
    surf_z = (-normal[0, 0]*surf_x - normal[0, 1]*surf_y - 0.5)* 1. / normal[0, 2]
    ax.plot_surface(surf_x, surf_y, surf_z, color=color, alpha=0.1)
    
def project2plane(data, normal, id_=0, color='green', shoot=False):
    pp = data[[id_]]
    proj = pp - np.sum((pp * normal)) * normal
    ax.scatter(*proj.ravel(), color=color, s=16)
    if shoot:
        kwargs = dict(
            color=color,  # colour of the curve
            linewidth=1.4,  # thickness of the line
            # linestyle = '--',  # available styles - -- -. :
            alpha=0.5,
        )
        ax.plot(*(zip(pp[0], proj[0])), **kwargs)
    return pp - np.sum(normal * pp) * normal  

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()
plot_vec(first, 6, alpha=0.4)
plot_plane(first, color='yellow', alpha=0.2, x_min=-2.5, x_max=2, y_min=-1, y_max=2)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_plane(first, color='yellow', alpha=0.2, x_min=-2.5, x_max=2, y_min=-1, y_max=2)

proj_on_plane=[]
for i in [5, 6]:
    aa= project2plane(data, first, id_=i, color='blue', shoot=True )
    proj_on_plane.append(aa)

    
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)
plot_plane(first, color='yellow', alpha=0.2, x_min=-2.5, x_max=2, y_min=-1, y_max=2)

for i in range(n):
    project2plane(data, first, id_=i, color='blue')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

plot_origin()
plot_plane(first, color='yellow', alpha=0.2, x_min=-2.5, x_max=2, y_min=-1, y_max=2)
plot_vec(second, 1.9, alpha=1, color='orange')

for i in range(n):
    project2plane(data, first, id_=i, color='blue')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

- This process can be continued to retain more and more information from the raw data. However, we remove one dimension each time when we remove the information recorded in a principal direction. 
- We cannot have more number of the principal components than the number of the original features we have. 
- This induction process always terminates within a finite step. 


In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
plot_origin()
plot_vec(first, 6, color='blue', alpha=1)
plot_vec(second, 1.9, color='orange', alpha=1)
plot_vec(third, 1.2, color='m', alpha=1)

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
plt.show()

### The Mathematical Formulation

In this section we identify all the elements we saw in the visualization with the mathematical formula. It is a good transition between the visualization and the Python code. To be compatible with the notation in Python, vectors are row vectors below.

- The first (very important) step we need to do is to centralize the data. We may then assume our data $X$ is an `n` by `p` matrix (that means we have a data set of `n` observations and `p` features). The average of each column is 0.
- We then project the data into any possible direction. A direction is represented by a unit vector $\hat{u}$ in linear algebra, and the projection is $ X \hat{u}^{\text{T}} $

**Question** What is the dimension of $\hat{u}$ ? What is the dimensions of $X \hat{u}^{\text{T}}$ ? Why do these dimension make sense?

1. We need to find the direction on which the projection of the data **spread most widely**. In the mathematical formulation, it can be stated as:
$$
\text{maximize Var} ( X \hat{u}^{\text{T}} )\\
\text{subject to } \| \hat{u} \| = 1
$$
    - The solution to the optimization problem above is the **first principal component direction**, denoted by $ \phi_1 $. The projection of our data $X$ on the first principal component direction is $ Z_1 = X \phi_1^{\text{T}}$, which is called the **first principal component**.

2.  Once the first $k-1$ principal components are found, the next one (if there is one) can be found inductively.
    - We first remove the information about the first k-1 components from $X$ ($X_k$ denotes the resulted matrix). 
$$
X_k =  X - \sum_{i=1}^{k-1} X \phi_i \phi_i^{\text{T}}
$$
    - With this matrix we solve the optimization problem again:
$$
\text{maximize Var} ( X_k \hat{u}^{\text{T}} )\\
\text{subject to } \| \hat{u} \| = 1
$$
    - Again the solution $\phi_k$ is the **kth principal component direction** and the projection on this direction, $Z_k = X_k \hat{u}^{\text{T}}$, is called the **kth principal component**.
    - **Note** Solving an optimization problem can be hard. In the setting of PCA, this is relatively easy. The principal component directions are (essentially) the **eigenvectors** of the covariance matrix of the data, arranged in the descending order of the eigenvalues they correspond to.

### PCA Properties
1. There are most `min(n, p)` principal components (but we often assume `p` of them).
2. The variances of each principal components decreases:
$$
\text{Var}(Z_1) ≥ \text{Var}(Z_2) ≥ ... ≥ \text{Var}(Z_p)
$$
3. The principal components $Z_1, Z_2, ..., Z_p$ are mutually uncorrelated.
4. The principal component directions
$
\phi_1, \phi_2, ..., \phi_p
$
are normalized and mutually perpendicular.

### Geometrical Meaning
- Geometrically we can imagine that the original data set sits inside $\mathbb{R}^f$ as a high dimensional scatterplot. 
- The selection of the top p principal directions establishes a linear projection $\mathbb{R}^f\to\mathbb{R}^p$ into a lower dimensional space. 
- There are many orthogonal linear projections $\mathbb{R}^f\to\mathbb{R}^p$. But PCA is special that it collapses directions in which the variances are small and preserve those whose variances are larger. 
- Those directions which get collapsed are interpreted as noise of the data. 
- Even though the apparent dimension of the data is f dimensional, PCA hypothesizes that the true dimension of the data lies in a p dimensional linear space. 
- In this sense, PCA is a de-noising process revealing the true nature of the data. 
- Nonlinear projections into curved objects instead of linear spaces is called manifold learning as non-linear smooth objects are called manifolds in geometry.  



In [ ]:
from sklearn.decomposition import PCA
pca = PCA()

**Arguments**: 

- **n_components**: Number of components to keep. In default it is `min(`n_samples, n_features`)`.

**Attributes**:

- **components_**: Components with maximum variance.
- **explained\_variance\_ratio\_**: Percentage of variance explained by each of the selected components. 
- **mean_**: The average of each feature.

**Methods:**

- **fit**: Fit the model with X.
- **fit_transform**: Fit the model with X and apply the dimensionality reduction on X.
- **inverse_transform**: Transform data back to its original space.
- **get_covariance**: Compute data covariance with the generative model.
- **get_params**: Get parameters for this estimator.
- **set_params**: Set the parameters of this estimator.
- **transform**: Apply the dimensionality reduction on X.

Here is a simple example. Let's use the data we visualize. For visualization, we have manually centralized the data. Here we use the data without being centralized, we will see that Python take care of this for us.

In [ ]:
n = 30
np.random.seed(108)
z = 10. * np.random.rand(n) - 5
theta = 2 * np.pi * np.random.rand(n)
a = 5 - np.abs(z)
x = a / 2.5 * np.cos(theta)
y = a / 5. * np.sin(theta)
w = a / 1.25 * np.sin(theta)
v = a / 0.75 * np.cos(theta)
data = np.zeros((n, 5))
data[:, 0] = x
data[:, 1] = y
data[:, 2] = z
data[:, 3] = w
data[:, 4] = v

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax1 = Axes3D(fig)

ax1.scatter(*data[:,3:5].T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
ax1.set_xlabel('z')
ax1.set_ylabel('w')
ax1.set_zlabel('v')
plt.show()

In [ ]:
pca.set_params(n_components=None)
pca.fit(data)
plt.plot(range(5), pca.explained_variance_ratio_)
plt.scatter(range(5), pca.explained_variance_ratio_)
plt.xlabel('ith components')
plt.ylabel('Percentage of Variance')
plt.show()

In [ ]:
pca.set_params(n_components=3).fit(data)
data2 = pca.transform(data)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
np.sum(pca.explained_variance_ratio_)

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = Axes3D(fig)

ax.scatter(*data2.T, marker='o', s=16, c='Lightgreen', edgecolors='k', alpha=1)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
plt.show()

In [ ]:
pca.set_params(n_components=2).fit(data)
data3 = pca.transform(data)

In [ ]:
plt.scatter(data3[:,0],data3[:,1])
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
pca.inverse_transform(data3) == data